# Compte rendu de R208-TP3 - Anonymisation de fichiers PCAP

## 1 - Fichier PCAP au format JSON

### 1.1 - La commande tshark (version texte de wireshark) permet d’afficher le contenu d’un fichier PCAP avec l’option -r.


In [ ]:
tshark -r tp3.pcap

### 1.2 - Utiliser en plus l’option -T pour avoir un affichage au format JSON « raw ». Créer alors le fichier tp3.json.

In [ ]:
tshark -T "json" -r tp3.pcap > tp3.json

### 1.3 - Créer alors un script python qui charge le fichier JSON et affiche le nombre de paquets.


In [3]:
import json

with open("./src/tp3.json", "rt", encoding="utf-8") as fin:
  file = json.loads(fin.read())
  packets_nbr = len(file)
  print(f"Il y a {packets_nbr} {'paquets' if packets_nbr > 1 else 'paquet'} dans ce fichier")

Il y a 41 paquets dans ce fichier


### 1.4 - Afficher ensuite la trame 14 (ARP) en hexadécimal en utilisant les données JSON. 

Attention, dans le  format JSON, les valeurs peuvent apparaître plusieurs fois.
Il faudra donc bien respecter le format d’une trame Ethernet contenant de l’ARP comme le précise wireshark :

Utiliser 2 listes (1 pour eth et 1 pour arp) contenant les clés à utiliser.

- Couche eth, 3 champs d’entête : @dst_mac, @src_mac, type
- Couche arp, 9 champs : hw_type, proto_type, hw_size, proto_size, opcode,  @src_hw_mac, @src_ipv4, @dst_hw_mac, @dst_ipv4
On doit obtenir au final :
  b07b25269ad93c5282ea5bd2080600010800060400013c5282ea5bd20acb00660000000000000acb0501

In [85]:
import json
import struct
import ipaddress

with open("./src/tp3.json", "rt", encoding="utf-8") as fin:
	packet = json.loads(fin.read())[13]

	eth_line = packet["_source"]["layers"]["eth"]
	arp_line = packet["_source"]["layers"]["arp"]

	eth_key = ["eth.dst_raw", "eth.src_raw", "eth.type_raw"]
	arp_key = ["arp.hw.type_raw", "arp.proto.type_raw", "arp.hw.size_raw", "arp.proto.size_raw", "arp.opcode_raw",
         "arp.src.hw_mac_raw", "arp.src.proto_ipv4_raw", "arp.dst.hw_mac_raw", "arp.dst.proto_ipv4_raw"]

	eth_data = [eth_line[key][0] for key in eth_key]
	arp_data = [arp_line[key][0] for key in arp_key]

	hex_data = "".join(eth_data + arp_data)

	print(True if hex_data == "b07b25269ad93c5282ea5bd2080600010800060400013c5282ea5bd20acb00660000000000000acb0501" else False)



True


## 2 - Création d’un fichier PCAP à partir d’une trame en hexadécimal :

### 2.1 - Pour générer un fichier PCAP lisible par wireshark/tshark, il faut en premier écrire l’entête PCAP:

```py
from struct import pack
pcap=open('arp-cb.pcap','wb')
pcap_hdr=pack('=IHHiIII',
               0xA1B2C3D4,  # magic_number
               2,           # version_major
               4,           # version_minor
               0,           # timezone
               0,           # sigfigs
               0x40000,     # snaplen
               1)           # ethernet
pcap.write(pcap_hdr)
```


### 2.2 - Ensuite, suivant le temps d’arrivée et la taille du paquet, on doit écrire l’entête spécifique pour le paquet puis le paquet :

```py
pkt_hexa='b07b25269ad93c5282ea5bd2080600010800060400013c5282ea5bd20acb00660000000000000acb0501'
ts_sec,ts_usec,pkt_len=0,0,len(pkt_hexa)//2
inc_len=pkt_len if pkt_len <= 65535 else 65535
pkt_hdr=pack('=IIII',
               ts_sec,      # ts_sec
               ts_usec,     # ts_usec
               inc_len,     # number of octets of packet saved in file
               pkt_len)     # actual length of packet
pcap.write(pkt_hdr)
pcap.write(bytes.fromhex(pkt_hexa))
pcap.close()
```

### 2.3 Vérifier alors que le fichier généré est bien valide et qu’il contient bien les données de la trame 14.

In [87]:
from struct import pack
pcap = open('arp-cb.pcap', 'wb')
pcap_hdr = pack('=IHHiIII',
                0xA1B2C3D4,  # magic_number
                2,           # version_major
                4,           # version_minor
                0,           # timezone
                0,           # sigfigs
                0x40000,     # snaplen
                1)           # ethernet
pcap.write(pcap_hdr)
pkt_hexa ='b07b25269ad93c5282ea5bd2080600010800060400013c5282ea5bd20acb00660000000000000acb0501'
ts_sec, ts_usec, pkt_len = 0, 0, len(pkt_hexa)//2
inc_len = pkt_len if pkt_len <= 65535 else 65535
pkt_hdr = pack('=IIII',
               ts_sec,      # ts_sec
               ts_usec,     # ts_usec
               inc_len,     # number of octets of packet saved in file
               pkt_len)     # actual length of packet
pcap.write(pkt_hdr)
pcap.write(bytes.fromhex(pkt_hexa))
pcap.close()


def isCorrect(file):
  with open("./src/tp3.json", "rt", encoding="utf-8") as fin:
    packet = json.loads(fin.read())[13]

    eth_line = packet["_source"]["layers"]["eth"]
    arp_line = packet["_source"]["layers"]["arp"]

    eth_key = ["eth.dst_raw", "eth.src_raw", "eth.type_raw"]
    arp_key = ["arp.hw.type_raw", "arp.proto.type_raw", "arp.hw.size_raw", "arp.proto.size_raw", "arp.opcode_raw",
              "arp.src.hw_mac_raw", "arp.src.proto_ipv4_raw", "arp.dst.hw_mac_raw", "arp.dst.proto_ipv4_raw"]

    eth_data = [eth_line[key][0] for key in eth_key]
    arp_data = [arp_line[key][0] for key in arp_key]

    hex_data = "".join(eth_data + arp_data)

    return True if hex_data == "b07b25269ad93c5282ea5bd2080600010800060400013c5282ea5bd20acb00660000000000000acb0501" else False

print(isCorrect("tp3_1.json"))


True


## 3 - Anonymisation de trames ARP et ICMP :

### 3.1 - On désire maintenant anonymiser les adresses MAC de l’entête Ethernet de la trame 14 en remplaçant les 3 derniers octets par un compteur.

Utiliser un dictionnaire (par exemple ethAddr) pour sauvegarder automatiquement la transformation des adresses MAC. On doit obtenir la trame suivante :

NB : On pourra créer les fonctions
`getARPHexaFromJSON1(ethAddr,pkts_json,frameNum)` et
`savePCAP1(fileName,kt_hexa)` :

```py
js=open('tp3.json').read()
pkts_json=json.loads(js)
ethAddr={}
pkt_hexa=getARPHexaFromJSON1(ethAddr,pkts_json,14)
savePCAP1('arp-anon1.pcap',pkt_hexa)
```

In [5]:

import json


def getHexHash(json_record, rawmode=True)-> str:
  eth_line = json_record["_source"]["layers"]["eth"]
  arp_line = json_record["_source"]["layers"]["arp"]

  eth_key = ["eth.dst", "eth.src", "eth.type"]
  arp_key = ["arp.hw.type_raw", "arp.proto.type_raw", "arp.hw.size_raw", "arp.proto.size_raw", "arp.opcode_raw",
             "arp.src.hw_mac_raw", "arp.src.proto_ipv4_raw", "arp.dst.hw_mac_raw", "arp.dst.proto_ipv4_raw"]


  if rawmode:
    eth_key = [key+"_raw" for key in eth_key]
    arp_key = [key+"_raw" for key in arp_key]

  eth_data = [eth_line[key][0] for key in eth_key]
  arp_data = [arp_line[key][0] for key in arp_key]

  hex_data = "".join(eth_data + arp_data)

def getArpHexaFromJson(eth_addr: str, pkts_json: dict, frame_nbr: int) -> str:
  arp_line = pkts_json[frame_nbr]["_source"]["layers"]["arp"]

  eth_addr.update({
    arp_line["arp.dst.hw_mac_raw"][0]: None,
    arp_line["arp.src.hw_mac_raw"][0]: None,
  })

  return None


def savePCAP(filename, pkt_hexa):
    if pkt_hexa == None:
       return

    from struct import pack
    pcap = open(filename, 'wb')
    pcap_hdr = pack('=IHHiIII',
                    0xA1B2C3D4,  # magic_number
                    2,  # version_major
                    4,  # version_minor
                    0,  # timezone
                    0,  # sigfigs
                    0x40000,  # snaplen
                    1)  # ethernet
    pcap.write(pcap_hdr)

    ts_sec, ts_usec, pkt_len = 0, 0, len(pkt_hexa)//2
    inc_len = pkt_len if pkt_len <= 65535 else 65535
    pkt_hdr = pack('=IIII',
                   ts_sec,  # ts_sec
                   ts_usec,  # ts_usec
                   inc_len,  # number of octets of packet saved in file
                   pkt_len)  # actual length of packet
    pcap.write(pkt_hdr)
    pcap.write(bytes.fromhex(pkt_hexa))
    pcap.close()



with open("./src/tp3.json", "rt", encoding="utf-8") as js:
  pkts_json = json.loads(js.read())
  eth_addr = {}
  pkt_hexa = getArpHexaFromJson(eth_addr, pkts_json, 14)
  savePCAP('arp-anon1.pcap', pkt_hexa)


### 3.2 - Comment faire pour anonymiser également les adresses MAC de la couche ARP (fonction `getARPHexaFromJSON2()`), sauf pour la valeur `000000000000` qui doit rester inchangée ?

On doit obtenir la trame suivante :

### 3.3 - Transformer maintenant tous les paquets ARP avec cette méthode (`fonctions savePCAP3()` et `getARPHexaFromJSON3(ethAddr,p)`).

On pourra tester la présence de la couche ARP dans les paquets avant de les traiter.  
L’adresse MAC de broadcast doit aussi rester inchangée.

### 3.4 - Comment changer les adresses IPv4 sur le même principe que pour les adresses MAC ?

### 3.5 - Anonymiser enfin les paquets ICMP en modifiant les adresses MAC de la couche Ethernet et les adresses IP de la couche IPv4.

ATTENTION, dans l’entête IPv4, les drapeaux et le décalage de fragment forment 16 bits donc il faut compléter avec des 0 la valeur de ip.frag_offset_raw.
On pourra utiliser les listes de champs JSON ci-dessous pour la génération de l’hexadécimal :

```py
ipv4= [ 'ip.version_raw', 'ip.dsfield_raw', 'ip.len_raw',
        'ip.id_raw', 'ip.flags_raw', 'ip.frag_offset_raw',
        'ip.ttl_raw', 'ip.proto_raw', 'ip.checksum_raw',
        'ip.src_raw', 'ip.dst_raw'
      ]
icmp= [ 'icmp.type_raw', 'icmp.code_raw', 'icmp.checksum_raw',
        'icmp.ident_raw', 'icmp.seq_raw', 'icmp.data_time_raw', 'data_raw'
      ]
```

### 3.6 - Activer la vérification du checksum dans les préférences du protocole IPv4 de wireshark, et constater que les paquets ICMP anonymisés ne sont plus valides.

Corriger alors les entêtes IPv4 en recalculant le checksum (voir la RFC791 page 14 ou bien par exemple https://en.wikipedia.org/wiki/IPv4_header_checksum).  
NB : en Python l’opérateur >> permet de décaler à droite (en binaire) un entier (division par les puissance de 
2), l’opérateur & permet de faire un ET bit à bit et l’opérateur ~ donne le complément à 1.